<a href="https://colab.research.google.com/github/gordonworldlee/ReinforcementLearning/blob/main/Copy_of_lebrodel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nba_api --upgrade

In [ ]:
# Import necessary libraries
import importlib
import inspect
from nba_api.stats.static import players, teams
from nba_api.stats.endpoints import ScoreboardV2  # Explicitly import ScoreboardV2
from tqdm import tqdm
import json
import pandas as pd
import logging
import time
from datetime import datetime, timedelta

In [ ]:
# Configure logging to capture errors and warnings
logging.basicConfig(
    filename='data_fetching_errors.log',
    level=logging.ERROR,  # Capture ERROR and above levels
    format='%(asctime)s:%(levelname)s:%(message)s'
)

In [ ]:
# Define All Endpoints
endpoints_list = [
    'AllTimeLeadersGrids',
    'AssistLeaders',
    'AssistTracker',
    'BoxScoreAdvancedV2',
    'BoxScoreFourFactorsV2',
    'BoxScoreMatchupsV3',
    'BoxScoreMiscV2',
    'BoxScorePlayerTrackV2',
    'BoxScoreScoringV2',
    'BoxScoreSummaryV2',
    'BoxScoreTraditionalV2',
    'BoxScoreUsageV2',
    'CommonAllPlayers',
    'CommonPlayerInfo',
    'CommonPlayoffSeries',
    'CommonTeamRoster',
    'CommonTeamYears',
    'CumeStatsPlayer',
    'CumeStatsPlayerGames',
    'CumeStatsTeam',
    'CumeStatsTeamGames',
    'DefenseHub',
    'DraftBoard',
    'DraftCombineDrillResults',
    'DraftCombineNonStationaryShooting',
    'DraftCombinePlayerAnthro',
    'DraftCombineSpotShooting',
    'DraftCombineStats',
    'DraftHistory',
    'FantasyWidget',
    'FranchiseHistory',
    'FranchiseLeaders',
    'FranchisePlayers',
    'GameRotation',
    'GLAlumBoxScoreSimilarityScore',
    'HomePageLeaders',
    'HomePageV2',
    'HustleStatsBoxScore',
    'InfographicFanDuelPlayer',
    'LeadersTiles',
    'LeagueDashLineups',
    'LeagueDashPlayerBioStats',
    'LeagueDashPlayerClutch',
    'LeagueDashOppPtShot',
    'LeagueDashPlayerPtShot',
    'LeagueDashPlayerShotLocations',
    'LeagueDashPlayerStats',
    'LeagueDashPtDefend',
    'LeagueDashPtStats',
    'LeagueDashPtTeamDefend',
    'LeagueDashTeamClutch',
    'LeagueDashTeamPtShot',
    'LeagueDashTeamShotLocations',
    'LeagueDashTeamStats',
    'LeagueHustleStatsPlayer',
    'LeagueHustleStatsTeam',
    'LeagueGameFinder',
    'LeagueGameLog',
    'LeagueLeaders',
    'LeagueLineupViz',
    'LeaguePlayerOnDetails',
    'LeagueSeasonMatchups',
    'LeagueStandings',
    'LeagueStandingsV3',
    'MatchupsRollup',
    'PlayByPlay',
    'PlayByPlayV2',
    'PlayerAwards',
    'PlayerCareerByCollege',
    'PlayerCareerByCollegeRollup',
    'PlayerCareerStats',
    'PlayerCompare',
    'PlayerDashPtPass',
    'PlayerDashPtReb',
    'PlayerDashPtShotDefend',
    'PlayerDashPtShots',
    'PlayerDashboardByClutch',
    'PlayerDashboardByGameSplits',
    'PlayerDashboardByGeneralSplits',
    'PlayerDashboardByLastNGames',
    'PlayerDashboardByShootingSplits',
    'PlayerDashboardByTeamPerformance',
    'PlayerDashboardByYearOverYear',
    'PlayerEstimatedMetrics',
    'PlayerFantasyProfile',
    'PlayerFantasyProfileBarGraph',
    'PlayerGameLog',
    'PlayerGameLogs',
    'PlayerGameStreakFinder',
    'PlayerNextNGames',
    'PlayerProfileV2',
    'PlayerVsPlayer',
    'PlayoffPicture',
    'ScoreboardV2',
    'ShotChartDetail',
    'ShotChartLeagueWide',
    'ShotChartLineupDetail',
    'SynergyPlayTypes',
    'TeamAndPlayersVsPlayers',
    'TeamDashLineups',
    'TeamDashPtPass',
    'TeamDashPtReb',
    'TeamDashPtShots',
    'TeamDashboardByGeneralSplits',
    'TeamDashboardByShootingSplits',
    'TeamDetails',
    'TeamEstimatedMetrics',
    'TeamGameLog',
    'TeamGameLogs',
    'TeamGameStreakFinder',
    'TeamHistoricalLeaders',
    'TeamInfoCommon',
    'TeamPlayerDashboard',
    'TeamPlayerOnOffDetails',
    'TeamPlayerOnOffSummary',
    'TeamVsPlayer',
    'TeamYearByYearStats',
    'VideoDetails',
    'VideoDetailsAsset',
    'VideoEvents',
    'VideoStatus',
    'WinProbabilityPBP'
]

In [ ]:
def inspect_and_populate_endpoints(endpoints):
    """
    Inspects NBA API endpoints to extract their constructor parameters and populates them with appropriate values.

    Parameters:
    - endpoints (list): A list of endpoint class names as strings.

    Returns:
    - dict: A dictionary mapping each endpoint to its populated parameters.
    """
    endpoint_parameters = {}
    importlib.invalidate_caches()

    # Fetch all active players and teams to use their IDs
    all_players = players.get_players()
    active_players = [player for player in all_players if player['is_active']]
    active_player_ids = [player['id'] for player in active_players]

    all_teams = teams.get_teams()
    active_team_ids = [team['id'] for team in all_teams]

    for endpoint in tqdm(endpoints, desc="Inspecting and Populating Endpoints"):
        try:
            # Dynamically import the endpoint module
            module = importlib.import_module(f'nba_api.stats.endpoints.{endpoint.lower()}')

            # Retrieve all classes in the module
            classes = inspect.getmembers(module, inspect.isclass)

            # Find the class that is defined in this module
            endpoint_class = None
            for cls_name, cls_obj in classes:
                if cls_obj.__module__ == module.__name__:
                    endpoint_class = cls_obj
                    break

            if not endpoint_class:
                print(f"No class found for endpoint: {endpoint}")
                continue

            # Get the constructor signature
            signature = inspect.signature(endpoint_class.__init__)

            # Extract parameters excluding 'self'
            params = {}
            for name, param in signature.parameters.items():
                if name == 'self':
                    continue
                # Assign parameters based on their names
                if 'player_id_list' in name or 'player_ids' in name:
                    params[name] = active_player_ids  # Assign all active player IDs
                elif 'vs_player_id_list' in name or 'vs_player_ids' in name:
                    params[name] = active_player_ids  # Assign all active player IDs for comparison
                elif 'team_id' in name and 'vs' not in name:
                    params[name] = active_team_ids  # Assign all active team IDs
                elif 'vs_team_id' in name or 'vs_team_ids' in name:
                    params[name] = active_team_ids  # Assign all active team IDs for comparison
                elif 'league_id' in name:
                    params[name] = '00'  # Assign league ID for NBA
                elif 'season_type_nullable' in name:
                    params[name] = 'Regular Season'  # Assign season type
                elif 'last_n_games' in name:
                    params[name] = 10  # Assign last N games as integer
                elif 'measure_type' in name:
                    params[name] = 'Base'  # Assign measure type
                elif 'month_nullable' in name:
                    params[name] = 0  # Assign month (0 for all)
                elif 'opponent_team_id_nullable' in name:
                    params[name] = 0  # Assign opponent team ID (0 for all)
                elif 'pace_adjust' in name:
                    params[name] = 'N'  # Assign pace adjustment
                elif 'per_mode_nullable' in name:
                    params[name] = 'Totals'  # Assign per mode
                elif 'period_nullable' in name:
                    params[name] = 0  # Assign period (0 for all)
                elif 'plus_minus_nullable' in name:
                    params[name] = 'N'  # Assign plus_minus
                elif 'rank_nullable' in name:
                    params[name] = 'N'  # Assign rank
                elif 'conference_nullable' in name:
                    params[name] = ''  # Assign conference (empty for all)
                elif 'date_from_nullable' in name:
                    params[name] = ''  # Assign start date (empty for all)
                elif 'date_to_nullable' in name:
                    params[name] = ''  # Assign end date (empty for all)
                elif 'division_simple_nullable' in name:
                    params[name] = ''  # Assign division (empty for all)
                elif 'game_segment_nullable' in name:
                    params[name] = ''  # Assign game segment (empty for all)
                elif 'location_nullable' in name:
                    params[name] = ''  # Assign location (empty for all)
                elif 'outcome_nullable' in name:
                    params[name] = ''  # Assign outcome (empty for all)
                elif 'season_segment_nullable' in name:
                    params[name] = ''  # Assign season segment (empty for all)
                elif 'shot_clock_range_nullable' in name:
                    params[name] = ''  # Assign shot clock range (empty for all)
                elif 'vs_conference_nullable' in name:
                    params[name] = ''  # Assign vs conference (empty for all)
                elif 'vs_division_nullable' in name:
                    params[name] = ''  # Assign vs division (empty for all)
                elif 'game_id' in name:
                    params[name] = None  # To be populated dynamically if needed
                elif 'player_id' in name and 'list' not in name:
                    params[name] = active_player_ids  # Assign all active player IDs; adjust as needed
                elif 'proxy' in name:
                    params[name] = None  # Assign proxy if needed
                elif 'headers' in name:
                    params[name] = None  # Assign headers if needed
                elif 'timeout' in name:
                    params[name] = 30  # Assign timeout
                elif 'get_request' in name:
                    params[name] = True  # Assign get_request
                else:
                    params[name] = None  # Default assignment for unspecified parameters

                # Handle any additional custom parameter assignments here as needed

            # Add to the mapping
            endpoint_parameters[endpoint] = params

        except ImportError:
            print(f"ImportError: Could not import endpoint '{endpoint}'. Please check the endpoint name or the nba_api version.")
            continue
        except Exception as e:
            print(f"Error inspecting endpoint '{endpoint}': {e}")
            continue

    # Convert the mapping to a pretty JSON string for readability
    endpoints_parameters_json = json.dumps(endpoint_parameters, indent=4)

    # Save the mapping to a JSON file
    with open('endpoints_parameters.json', 'w') as f:
        f.write(endpoints_parameters_json)

    print("\nEndpoint parameters have been successfully extracted and saved to 'endpoints_parameters.json'.")

    # Optionally, display parameters for a specific endpoint, e.g., 'PlayerCompare'
    endpoint_to_review = 'PlayerCompare'
    print(f"\nParameters for {endpoint_to_review}:")
    print(json.dumps(endpoint_parameters.get(endpoint_to_review, {}), indent=4))

    # Pause briefly to ensure file writing is complete
    time.sleep(2)

    # Load the parameters mapping from the JSON file
    with open('endpoints_parameters.json', 'r') as f:
        endpoints_parameters_loaded = json.load(f)

    return endpoints_parameters_loaded

In [ ]:
# Execute the inspection and populate parameters
endpoints_parameters_loaded = inspect_and_populate_endpoints(endpoints_list)

In [ ]:
def fetch_endpoint_data(endpoint_class, params):
    """
    Fetches data from an NBA API endpoint class with enhanced error handling.

    Parameters:
    - endpoint_class: The class of the NBA API endpoint.
    - params: A dictionary of parameters required by the endpoint.

    Returns:
    - A pandas DataFrame containing the fetched data or an empty DataFrame on failure.
    """
    try:
        # Remove parameters that are None and not required
        clean_params = {k: v for k, v in params.items() if v is not None}

        # Instantiate the endpoint with the provided parameters
        instance = endpoint_class(**clean_params)

        # Attempt to use get_data_frame() if it exists
        if hasattr(instance, 'get_data_frame') and callable(getattr(instance, 'get_data_frame')):
            df = instance.get_data_frame()
            if isinstance(df, pd.DataFrame) and not df.empty:
                return df
            else:
                logging.error(f"get_data_frame() did not return a valid DataFrame for {endpoint_class.__name__}")
                return pd.DataFrame()

        # Else, use get_data_frames() and concatenate if multiple DataFrames are returned
        elif hasattr(instance, 'get_data_frames') and callable(getattr(instance, 'get_data_frames')):
            df_list = instance.get_data_frames()
            if isinstance(df_list, list) and len(df_list) > 0:
                # Filter out empty DataFrames
                df_list = [df for df in df_list if not df.empty]
                if df_list:
                    combined_df = pd.concat(df_list, ignore_index=True)
                    return combined_df
                else:
                    logging.error(f"get_data_frames() returned all empty DataFrames for {endpoint_class.__name__}")
                    return pd.DataFrame()
            else:
                logging.error(f"get_data_frames() returned no DataFrames for {endpoint_class.__name__}")
                return pd.DataFrame()

        else:
            logging.error(f"No data fetching method found for {endpoint_class.__name__}")
            return pd.DataFrame()

    except Exception as e:
        logging.error(f"Error fetching data from {endpoint_class.__name__}: {e}")
        return pd.DataFrame()

In [ ]:
def get_recent_game_date(days_back=30):
    """
    Retrieves the most recent game date within the last 'days_back' days.

    Parameters:
    - days_back (int): Number of days back to search for the most recent game.

    Returns:
    - str or None: The most recent game date in 'YYYY-MM-DD' format or None if not found.
    """
    try:
        # Start from today and go back up to 'days_back' days to find a game date
        for i in range(days_back):
            game_date = (datetime.today() - timedelta(days=i)).strftime('%Y-%m-%d')
            # Initialize ScoreboardV2 with correct parameters
            scoreboard = ScoreboardV2(game_date=game_date, league_id='00')  # '00' for NBA
            games_df = scoreboard.get_data_frames()[0]
            if not games_df.empty:
                return game_date
        # If no games found in the last 'days_back' days, return None
        return None
    except Exception as e:
        logging.error(f"Error retrieving recent game date: {e}")
        return None

In [ ]:
# Initialize a dictionary to store fetched data
fetched_data = {}

# Retrieve the most recent game date
recent_game_date = get_recent_game_date(days_back=30)

if recent_game_date:
    print(f"Most recent game date found: {recent_game_date}")
else:
    print("No recent game date found. Some endpoints may fail due to missing 'game_id'.")

# Iterate through each endpoint and fetch data
for endpoint_name in tqdm(endpoints_list, desc="Fetching Data for Endpoints"):
    endpoint_class = None
    try:
        # Dynamically import the endpoint module
        module = importlib.import_module(f'nba_api.stats.endpoints.{endpoint_name.lower()}')

        # Retrieve all classes in the module
        classes = inspect.getmembers(module, inspect.isclass)

        # Find the class that is defined in this module
        for cls_name, cls_obj in classes:
            if cls_obj.__module__ == module.__name__:
                endpoint_class = cls_obj
                break

        if not endpoint_class:
            print(f"No class found for endpoint: {endpoint_name}")
            continue

        # Retrieve parameters for the endpoint
        params = endpoints_parameters_loaded.get(endpoint_name, {})

        # Handle specific endpoints that require dynamic parameter assignments
        # For example, endpoints that require specific game_ids or player comparisons

        # Example: For endpoints requiring 'game_id', fetch recent game IDs
        if 'game_id' in params and params['game_id'] is None:
            if recent_game_date:
                try:
                    # Fetch recent game IDs based on the most recent game date
                    scoreboard = ScoreboardV2(game_date=recent_game_date, league_id='00')
                    games_df = scoreboard.get_data_frames()[0]
                    if not games_df.empty:
                        recent_game_ids = games_df['GAME_ID'].tolist()
                        if recent_game_ids:
                            params['game_id'] = recent_game_ids[0]  # Assign the first recent game_id
                        else:
                            params['game_id'] = None  # No recent game found
                    else:
                        params['game_id'] = None  # No games on this date
                except Exception as e:
                    logging.error(f"Error fetching game_id for endpoint '{endpoint_name}': {e}")
                    params['game_id'] = None
            else:
                logging.error(f"No recent game date found to assign 'game_id' for endpoint '{endpoint_name}'.")
                params['game_id'] = None  # Handle accordingly

        # Example: For 'CumeStatsPlayer' and 'CumeStatsTeam', assign 'game_ids'
        if endpoint_name in ['CumeStatsPlayer', 'CumeStatsTeam']:
            if 'game_ids' in params and (params['game_ids'] is None or len(params['game_ids']) == 0):
                if recent_game_date:
                    try:
                        scoreboard = ScoreboardV2(game_date=recent_game_date, league_id='00')
                        games_df = scoreboard.get_data_frames()[0]
                        if not games_df.empty:
                            recent_game_ids = games_df['GAME_ID'].tolist()
                            if recent_game_ids:
                                params['game_ids'] = recent_game_ids  # Assign all recent game_ids
                            else:
                                params['game_ids'] = []  # No recent game found
                        else:
                            params['game_ids'] = []  # No games on this date
                    except Exception as e:
                        logging.error(f"Error fetching game_ids for endpoint '{endpoint_name}': {e}")
                        params['game_ids'] = []
                else:
                    logging.error(f"No recent game date found to assign 'game_ids' for endpoint '{endpoint_name}'.")
                    params['game_ids'] = []

        # Example: For 'PlayerCompare', ensure player lists are populated
        if endpoint_name == 'PlayerCompare':
            # Limit the number of players to top_n_players for comparison to avoid excessive data
            top_n = 10  # Adjust as needed
            params['player_id_list'] = params.get('player_id_list', [])[:top_n]  # Top N active players
            params['vs_player_id_list'] = params.get('vs_player_id_list', [])[:top_n]  # Comparing against top N active players

        # Update the parameters in the mapping
        endpoints_parameters_loaded[endpoint_name] = params

        # Fetch data using the fetch_endpoint_data function
        df = fetch_endpoint_data(endpoint_class, params)

        if not df.empty:
            fetched_data[endpoint_name] = df
            print(f"Successfully fetched data for endpoint: {endpoint_name}")
        else:
            print(f"No data fetched for endpoint: {endpoint_name}")

    except ImportError:
        print(f"ImportError: Could not import endpoint '{endpoint_name}'. Please check the endpoint name or the nba_api version.")
        continue
    except Exception as e:
        logging.error(f"Error processing endpoint '{endpoint_name}': {e}")
        print(f"Error processing endpoint '{endpoint_name}'. Check logs for details.")
        continue

NameError: name 'get_recent_game_date' is not defined